<a href="https://colab.research.google.com/github/PenroseTiles/Tensorflow2.0_Tutorials/blob/master/TF2_MNIST_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Heavily borrowed from tensorflow.org

In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import numpy as np
import tensorflow as tf
from tensorflow import keras

TensorFlow 2.x selected.


In [0]:
import typing

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Reshape
from tensorflow.keras import Model, Input

Testing eager execution, vetarans who worked with TF 1.x will know the pain of debugging without this

In [0]:
a = tf.constant(2.0)
b = tf.constant(3.0)
c = a+b

In [5]:
c.numpy()

5.0

#Load the dataset

In [0]:
mnist = keras.datasets.mnist

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

By using ellipsis, I avoid stating exactly the number of dimensions of the Tensor x_train, I just want to add a dimension to the tensor, regardless of how many dimensions it has

In [0]:
# x_train = x_train[..., tf.newaxis]
# x_test = x_test[..., tf.newaxis]

# Playing with *tf.Dataset*

In [9]:
data = tf.data.Dataset.from_tensor_slices(tf.range(8))
it = iter(data)
print(next(it).numpy())

for i in data:
  print(i.numpy())

0
0
1
2
3
4
5
6
7


Map and Reduce

In [10]:
@tf.function
def sum_data(data):
  return data.reduce(0, lambda state, value : state + value)

@tf.function
def transform_data(data: tf.data.Dataset):
  return data.map(lambda x : x+ 1)

sum_data(data).numpy()
mapped = transform_data(data) #

'''
mapped.numpy() will not work because mapped is an iterable, like every Dataset object
But if I can't call numpy() on Datasets, how do I convert them to non-tensor form? 
You can call as_numpy_iterator() on it, it basically does what we do below: 
'''
for item in mapped:
  print(item.numpy())

1
2
3
4
5
6
7
8


In [0]:
mn = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(2)
# mnist_y = tf.data.Dataset.from_tensor_slices(y_train)
# mnist = tf.data.Dataset.zip((mnist_data_x, mnist_data_y))
# mnist = mnist_x




In [12]:
for image, label in mn:
  print(image.numpy().shape)
  break

(2, 28, 28)


In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


In [14]:
for images, labels in train_ds:
    print(images.numpy().shape)
    break

(32, 28, 28)


# The model

In [15]:
inputs = Input(shape=(28,28))
x = Reshape(target_shape=(784,), input_shape=(28,28))(inputs)

f = Dense(units=400, activation='relu')(x)
y = Dense(units=10)(f)
model = keras.Model(inputs, y, name='mnist_nielsen')
model.summary()

Model: "mnist_nielsen"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
reshape (Reshape)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 400)               314000    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                4010      
Total params: 318,010
Trainable params: 318,010
Non-trainable params: 0
_________________________________________________________________


To calculate the loss, create a loss object (which is callable) and maintain a running average using tf.keras.metrics.Mean

To calculate accuracy, just create an accuracy object (which is callable) which is itself a metric, so we need not do anything further to track its running average.

The optimizer is not a callable object, instead you call the `apply_gradients` method

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

To calculate accuracy, just create an accuracy object (which is callable) which is itself a metric, so we need not do anything further to track its running average.

In [0]:
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Create separate metrics for train and test. 

In [0]:
train_loss_object = tf.keras.metrics.Mean('train_loss')
test_loss_object = tf.keras.metrics.Mean('test_loss')

In [0]:
def epoch(dataset, training=False):
  total_loss = 0
  loss_tracker = train_loss_object if training else test_loss_object
  # print("loss trackers created")
  acc_object = train_accuracy if training else test_accuracy
  for images, labels in dataset:
    with tf.GradientTape() as tape:
      prediction = model(images, training=training)
      # print(prediction.numpy().shape)
      loss = loss_object(labels, prediction)
      acc_object(labels, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  loss_tracker(loss)
  print(acc_object.result())
  return 

def train_epoch():
  epoch(train_ds, training=True)

def test_epoch():
  epoch(test_ds)


In [0]:
for epoch_number in range(20):
  train_epoch()

In [21]:
train_loss_object.reset_states()
train_loss_object.reset_states()
train_accuracy.reset_states()
test_accuracy.reset_states()

test_epoch()

tf.Tensor(0.9746, shape=(), dtype=float32)
